<a href="https://colab.research.google.com/github/JeongCheolHee/colab/blob/main/%ED%8A%B8%EB%A6%AC%EC%9D%98_%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#정형 데이터를 다루는 앙상블 학습

In [1]:
'''랜덤 포레스트
부트스트랩 샘플 : 복원추출로 샘플을 뽑은 것 
랜덤 포레스트는 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합되는 것을 막아주고
검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있다.'''

'랜덤 포레스트\n부트스트랩 샘플 : 복원추출로 샘플을 뽑은 것 \n랜덤 포레스트는 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합되는 것을 막아주고\n검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있다.'

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv("https://bit.ly/wine_csv_data")
print(wine.head())
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, 
                                                                      test_size = 0.2, random_state = 42)

   alcohol  sugar    pH  class
0      9.4    1.9  3.51    0.0
1      9.8    2.6  3.20    0.0
2      9.8    2.3  3.26    0.0
3      9.8    1.9  3.16    0.0
4      9.4    1.9  3.51    0.0


In [7]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1, random_state = 42) #n_jobs = -1로 지정하면 모든 cpu코어를 사용
scores = cross_validate(rf, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score'])) #return_train_score = True로 두면 훈련세트와 검증 세트점수 비교 용이

0.9973541965122431 0.8905151032797809


In [10]:
rf.fit(train_input, train_target)
print(rf.feature_importances_) #결정 트리는 특성 중요도를 계산해줌, 랜덤 포레스트는 특성의 일부를 램던하게 선택하여 결정 트리를 훈련하기 때문에 과대적합을 줄이고 일반화 성능을 높이는 데 도움

[0.23167441 0.50039841 0.26792718]


In [11]:
'''랜덤포레스트는 훈련세트에서 중복을 허용하여 부트스트랩 샘플을 만들 때 부트스트랩 샘플에 포함되지 않은
샘플들(OOB)을 통해 검증 세트의 역할을 할 수 있다'''
rf = RandomForestClassifier(oob_score = True, n_jobs = -1, random_state = 42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


In [12]:
#엑스트라 트리 
#각 결정 트리를 만들 때 전체 훈련 세트를 사용함, 노드를 분할할 때 무작위로 분할
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs= -1, random_state = 42)
scores = cross_validate(et, train_input, train_target,
                        return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))


0.9974503966084433 0.8887848893166506


In [14]:
#그레이디언트 부스팅
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state = 42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))


0.8881086892152563 0.8720430147331015


In [15]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate = 0.2,
                                random_state = 42) #결정 트리 개수를 500개로 늘려도 과대적합을 억제
scores = cross_validate(gb, train_input, train_target,
                        return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [16]:
#히스토그램 기반 그레이디언트 부스팅
#정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높은 알고리즘 
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state = 42)
scores = cross_validate(hgb, train_input, train_target,
                        return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

/usr/local/lib/python3.8/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.9321723946453317 0.8801241948619236


In [23]:
from pandas.core.common import random_state
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb,train_input, train_target, n_repeats=10, random_state = 42, n_jobs = -1)
print(result.importances_mean) #(특성의 중요도, 평균, 표준 편차)

result = permutation_importance(hgb,test_input, test_target, n_repeats=10, random_state = 42, n_jobs = -1)
print(result.importances_mean)
#성능 최종 확인
hgb.score(test_input, test_target)

[0.08876275 0.23438522 0.08027708]
[0.05969231 0.20238462 0.049     ]


0.8723076923076923

In [30]:
#XGBoost
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method = 'hist', random_state = 42)
scores = cross_validate(xgb, train_input, train_target, return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#LightGMB
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state = 42)
scores = cross_validate(lgb, train_input, train_target, return_train_score= True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284
0.9338079582727165 0.8789710890649293
